<a href="https://colab.research.google.com/github/mardzyuba/Faker/blob/master/lesson_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

# Задание 1.

In [3]:
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df = df.loc[df['Quantity'] > 0]
df = df.dropna(subset=['CustomerID'])

# Задание 2.

Создайте матрицу item-customer (по строкам - товары, по столбцам - покупатели)

In [5]:
StockCode_item_matrix = df.pivot_table(index='StockCode', columns='CustomerID', values='Quantity', aggfunc='sum')
StockCode_item_matrix.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,12357.0,12358.0,12359.0,12360.0,12361.0,12362.0,12363.0,12364.0,12365.0,12367.0,12370.0,12371.0,12372.0,12373.0,12374.0,12375.0,12377.0,12378.0,12379.0,12380.0,12381.0,12383.0,12384.0,12386.0,12388.0,12390.0,12391.0,12393.0,12394.0,12395.0,...,18230.0,18231.0,18232.0,18233.0,18235.0,18236.0,18237.0,18239.0,18240.0,18241.0,18242.0,18245.0,18246.0,18248.0,18249.0,18250.0,18251.0,18252.0,18255.0,18257.0,18259.0,18260.0,18261.0,18262.0,18263.0,18265.0,18268.0,18269.0,18270.0,18272.0,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
StockCode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
StockCode_item_matrix.shape

(3665, 4339)

In [7]:
df['CustomerID'].nunique()

4339

In [8]:
df['StockCode'].nunique()

3665

In [9]:
StockCode_item_matrix = StockCode_item_matrix.applymap(lambda x: 1 if x > 0 else 0)
StockCode_item_matrix.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,12357.0,12358.0,12359.0,12360.0,12361.0,12362.0,12363.0,12364.0,12365.0,12367.0,12370.0,12371.0,12372.0,12373.0,12374.0,12375.0,12377.0,12378.0,12379.0,12380.0,12381.0,12383.0,12384.0,12386.0,12388.0,12390.0,12391.0,12393.0,12394.0,12395.0,...,18230.0,18231.0,18232.0,18233.0,18235.0,18236.0,18237.0,18239.0,18240.0,18241.0,18242.0,18245.0,18246.0,18248.0,18249.0,18250.0,18251.0,18252.0,18255.0,18257.0,18259.0,18260.0,18261.0,18262.0,18263.0,18265.0,18268.0,18269.0,18270.0,18272.0,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
StockCode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10080,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10120,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10133,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Задание 3.

Проведите оценку мер близости товаров, получив матрицу item_item_sim_matrix со значениями косинусов между векторами товаров.

In [10]:
from scipy.sparse import csr_matrix

In [11]:
mat = StockCode_item_matrix.values
mat

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [12]:
def cos_sim(array):
  
  csr_array = csr_matrix(array)
  numerator = np.dot(csr_array,csr_array.T).toarray()
  denomenator = np.linalg.norm(array, axis = 1, keepdims = True)*np.linalg.norm(array.T, axis = 0, keepdims = True)
  return numerator / denomenator 

In [13]:
user_user_sim_matrix = cos_sim(mat)
user_user_sim_matrix

array([[1.        , 0.        , 0.09486833, ..., 0.06691496, 0.        ,
        0.07821652],
       [0.        , 1.        , 0.        , ..., 0.01618174, 0.        ,
        0.        ],
       [0.09486833, 0.        , 1.        , ..., 0.07053456, 0.        ,
        0.01099299],
       ...,
       [0.06691496, 0.01618174, 0.07053456, ..., 1.        , 0.        ,
        0.0775386 ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.07821652, 0.        , 0.01099299, ..., 0.0775386 , 0.        ,
        1.        ]])

In [14]:
user_user_sim_matrix = pd.DataFrame(user_user_sim_matrix)

user_user_sim_matrix.columns = StockCode_item_matrix.index

user_user_sim_matrix['StockCode'] = StockCode_item_matrix.index
user_user_sim_matrix = user_user_sim_matrix.set_index('StockCode')

user_user_sim_matrix.head()

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,15039,16008,16010,16011,16012,16014,16015,16016,16033,16043,16045,16046,16048,16049,16052,16054,16216,16218,16219,16225,16235,16236,16237,16238,16254,16259,17001,17003,17021,17038,...,90209B,90209C,90210A,90210B,90210C,90210D,90211A,90211B,90212B,90212C,90214A,90214B,90214C,90214D,90214E,90214F,90214G,90214H,90214I,90214J,90214K,90214L,90214M,90214N,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.090351,0.062932,0.098907,0.095346,0.047673,0.075593,0.090815,0.062284,0.043033,0.182574,0.076948,0.027524,0.054233,0.036274,0.049386,0.0,0.0,0.040489,0.043853,0.024693,0.000000,0.070711,0.058722,0.076948,0.072336,0.049386,0.062828,0.095871,0.102923,0.106012,0.080978,0.000000,0.119523,0.0,0.094907,0.115470,0.000000,...,0.0,0.055902,0.000000,0.000000,0.000000,0.000000,0.091287,0.0,0.0,0.0,0.000000,0.0,0.000000,0.070711,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029361,0.0,0.066915,0.000000,0.078217
10080,0.000000,1.000000,0.000000,0.032774,0.045655,0.047836,0.000000,0.000000,0.082261,0.049413,0.030124,0.062439,0.000000,0.037216,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.117495,0.000000,0.035829,0.102598,0.000000,0.000000,0.037216,0.069971,0.107486,0.060774,0.027821,0.029867,0.065922,0.058747,0.000000,0.043355,0.0,0.019672,0.041885,0.108148,...,0.0,0.081111,0.000000,0.000000,0.102598,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.162221,0.102598,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.016182,0.000000,0.000000
10120,0.094868,0.000000,1.000000,0.057143,0.059702,0.041703,0.060302,0.060302,0.095618,0.028718,0.026261,0.027217,0.115470,0.032444,0.139262,0.068599,0.045883,0.031235,0.0,0.0,0.076822,0.110940,0.031235,0.000000,0.000000,0.111417,0.162221,0.060999,0.062470,0.185435,0.121268,0.234340,0.344817,0.230466,0.000000,0.113389,0.0,0.051450,0.036515,0.047140,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.115470,0.0,0.0,0.0,0.000000,0.0,0.000000,0.089443,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.010993
10125,0.090351,0.032774,0.057143,1.000000,0.042644,0.044682,0.043073,0.000000,0.051224,0.030770,0.056274,0.077762,0.000000,0.069523,0.049736,0.024500,0.000000,0.000000,0.0,0.0,0.091455,0.039621,0.000000,0.063888,0.000000,0.053056,0.046349,0.000000,0.000000,0.018922,0.121268,0.130189,0.136832,0.109746,0.053995,0.053995,0.0,0.036750,0.000000,0.000000,...,0.0,0.000000,0.000000,0.058321,0.000000,0.071429,0.000000,0.0,0.0,0.0,0.063888,0.0,0.053995,0.063888,0.058321,0.000000,0.000000,0.0,0.0,0.0,0.058321,0.071429,0.0,0.0,0.0,0.071429,0.0,0.063888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.070669
10133,0.062932,0.045655,0.059702,0.042644,1.000000,0.280097,0.045002,0.060003,0.071358,0.057152,0.052262,0.121867,0.057448,0.080708,0.051964,0.068259,0.068483,0.093239,0.0,0.0,0.127401,0.082792,0.093239,0.000000,0.133498,0.203251,0.064567,0.121393,0.139859,0.065898,0.084466,0.116589,0.104838,0.101921,0.037609,0.094022,0.0,0.068259,0.054500,0.046907,...,0.0,0.035180,0.057448,0.040622,0.000000,0.000000,0.057448,0.0,0.0,0.0,0.000000,0.0,0.000000,0.044499,0.000000,0.070360,0.044499,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036955,0.0,0.070185,0.049752,0.021877


# Задание 4.

Отберите ТОП-10 похожих товаров по StockCode.
Возьмем товар 10002

In [15]:

top = user_user_sim_matrix.loc[10002].sort_values(ascending=False).head(10)

top

StockCode
10002     1.000000
90103     0.223607
21826     0.197642
90059E    0.182574
90059C    0.182574
90101     0.182574
90059B    0.182574
90059F    0.182574
16010     0.182574
84535A    0.181369
Name: 10002, dtype: float64

In [ ]:
items_bought_by_top = top[top>0].index.tolist()

In [18]:
df.loc[df['StockCode'].isin(items_bought_by_top ), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode')

,Description
StockCode,
10002,INFLATABLE POLITICAL GLOBE
90059B,DIAMANTE HAIR GRIP PACK/2 BLACK DIA
90059E,DIAMANTE HAIR GRIP PACK/2 RUBY
90059C,DIAMANTE HAIR GRIP PACK/2 MONTANA
21826,EIGHT PIECE DINOSAUR SET
16010,FOLDING CAMPING SCISSOR W/KNIF & S
84535A,ENGLISH ROSE NOTEBOOK A6 SIZE
90059F,DIAMANTE HAIR GRIP PACK/2 LT ROSE
90101,WHITE FRANGIPANI NECKLACE
